# midiAI

In this notebook, I'll go through my process of using `music21` to create lists of information about songs generated from the `.midi` files in `data/` generated from the [*OpenBook*](https://veltzer.github.io/openbook/) repository.

Let's begin by importing our libraries. We're going to be using `midi2audio` to playback our generated `.wav`.

In [20]:
from IPython.display import Audio
from midi2audio import FluidSynth
from music21 import converter, harmony, pitch, chord, key, interval, note
from pathlib import Path
from pprint import pprint

import subprocess

## `.midi` file playback

Let's get our playback system running by defining our `FluidSynth` using the `gm-soundfont.sf2` from the project.

In [21]:
fs = FluidSynth(sound_font="assets/gm-soundfont.sf2")

Let's make it easier to create `.wav` files for playback with this quick `.midi` to `.wav` function that creates the `.wav` file and `return`s the output filename.

We're also going to need to hide the wildly long fluidsynth warning outputs, so we'll do that here.

In [22]:
def play_midi(mid_path):
	output_dir = Path("out")
	output_dir.mkdir(exist_ok=True)
	output_name = output_dir / Path(mid_path).with_suffix(".wav").name
	
	# fs.midi_to_audio(mid_path, output_name)
	subprocess.run(
		["fluidsynth", "-ni", "gm-soundfont.sf2", str(mid_path), "-F", str(output_name), "-r 22050", "-q"],
		stdout=subprocess.DEVNULL,
		stderr=subprocess.DEVNULL
	)

	return output_name

As a test, we're going to work with `solar.midi` from `data/`.

In [23]:
solar_path = "data/solar.midi"

Now we can give our `play_midi` function a go and make sure it's working. If you're following along with this notebook, make sure `fluidsynth` is installed using `brew install fluidsynth` if on a Mac.

In [24]:
# Audio(play_midi(solar_path))

## `.music21` parsing

Let's get into using `music21` to sort through our `.midi` file.

In [25]:
solar = converter.parse(solar_path)

We'll inevitably come into problems trying to analyze the `harmony.chordSymbolFromChord` when `music21` can't figure out what the chord symbol should be because there are sharp symbols instead of flat symbols, so let's prepare a couple of `dict`s to replace those notes if we come across them.

In [26]:
note_replacements = {
	"A#": "B-", "C#": "D-", "D#": "E-", "F#": "G-", "G#": "A-"
}

note_replacements_reverse = {
	"B-": "A#", "D-": "C#", "E-": "D#", "G-": "F#", "A-": "G#"
}

### `chord_measures`

Now let's get into creating lists of chord names and note information from each measure. The chords will always be in the first part of the parsed `.midi` file (index 0) and the notes will always be the second part (index 1).

We'll then create empty lists that will hold each measure of chord(s) and note information. You'll notice I had to replace a few sharp notes with flats in order to get the correct chords as well as completely replacing the chord name for a G7b9, which was showing up as G7addG# (essentially the same thing).

### `note_measures`

The note information will be stored in the `note_measures` list where each note is a list with the note's *name* and *octave* in one string and the length of the note (in terms of `music21`'s `quarterNote`) in the next element of the list.

In [27]:
chords = solar.parts[0]
melody = solar.parts[1]

chord_measures = []
for item in chords:
	measure = []
	for c in item:
		if "chord.Chord" in str(c):
			cs = harmony.chordSymbolFromChord(c)
			if "Cannot" in cs.figure:
				pitches = [p for p in c.pitches]
				for idx, p in enumerate(pitches):
					if p.accidental is not None and "#" in p.name:
						pitches[idx] = pitch.Pitch(note_replacements[p.name], octave=p.octave)
				new_cs = harmony.chordSymbolFromChord(chord.Chord(pitches))
				measure.append(new_cs.figure)
			else:
				if f"add{cs.figure[0]}#" in cs.figure:
					nc = cs.figure.replace(f"add{cs.figure[0]}#", "b9")
					measure.append(nc)
				else:
					measure.append(cs.figure)
	chord_measures.append(measure)

note_measures = []
for item in melody:
	measure = []
	for n in item:
		if "note.Note" in str(n) or "note.Rest" in str(n):
			n_name = n.nameWithOctave if "note.Note" in str(n) else "RS"
			n_len = n.quarterLength
			measure.append([n_name, n_len])
	note_measures.append(measure)

Let's take a look at measure one of `chord_measures` and `note_measures`.

In [28]:
print(chord_measures[0])
print(note_measures[0])

['Cm']
[['RS', 0.5], ['C5', 1.5], ['B4', 1.0], ['D5', 0.5], ['C5', 0.5]]


## Sidestep, transposing

Let's say we have a song in the key of Eb major. For training purposes, I want to only train and generate songs in C major. Can we transpose using `music21`?

In [29]:
misty_path = "data/misty.midi"

In [30]:
# Audio(play_midi(misty_path))

In [31]:
misty = converter.parse(misty_path)

Below, I show the route I went with to convert a `.midi` file to C major/A minor. For this purpose, I also exported it as a `.mid` file for playback so you can compare with the previous one.

In [ ]:
misty_key = misty.flat.getElementsByClass(key.KeySignature)[0]
target_key = key.Key("C") if misty_key.mode == "major" else key.Key("A", "minor")
i = interval.Interval(misty_key.tonic, target_key.tonic)

misty_in_c = misty.transpose(i)
# misty_in_c.write("midi", fp="misty_in_c.mid")
# Audio(play_midi("misty_in_c.mid"))

c:\Users\circ8\Documents\Chris\GitHub Repositories\midi-ai\.venv\Lib\site-packages\music21\stream\base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


'misty_in_c.mid'

So now that we know how to make sure we have consistent data for training, we can get back to where we were.

## Loops -> functions

Let's make functions out of our loops from before, as well as whatever else we can, to make things easier in the future.

In [ ]:
harmony.addNewChordSymbol("7alt", "1,-3,-5,-7,-9,-11,-13", ["alt7", "7alt"])
harmony.addNewChordSymbol("m6", "1,-3,5,6", ["m6", "(min6)"])
harmony.addNewChordSymbol("13", "1,3,5,7,9,13", ["13", "(13)"])
harmony.addNewChordSymbol("69", "1,3,5,6,9", ["69", "6/9"])

custom_chords = ["7alt", "m6", "13", "69"]

def is_custom_chord(input_chord):
	root = input_chord.pitches[0].name
	custom_idx = [False] * len(custom_chords)
	for p in [p.name for p in input_chord.pitches]:
		for idx, c_name in enumerate(custom_chords):
			c_chord = harmony.ChordSymbol(f"{root}{c_name}")
			if p in [c.name for c in c_chord.pitches]:
				custom_idx[idx] = True
			else:
				custom_idx[idx] = False

	if True in custom_idx:
		return True
	else:
		return False

def to_custom_chord(input_chord):
	root = input_chord.pitches[0].name
	custom_idx = [False] * len(custom_chords)
	for p in [p.name for p in input_chord.pitches]:
		for idx, c_name in enumerate(custom_chords):
			c_chord = harmony.ChordSymbol(f"{root}{c_name}")
			if p in [c.name for c in c_chord.pitches]:
				custom_idx[idx] = True
			else:
				custom_idx[idx] = False

	if True in custom_idx:
		for idx, c in enumerate(custom_idx):
			if c:
				return harmony.ChordSymbol(f"{root}{custom_chords[idx]}")
	else:
		return input_chord

def gen_chords(chords_part):
	chord_measures = []
	for item in chords_part:
		measure = []
		for c in item:
			if "chord.Chord" in str(c):
				if is_custom_chord(c):
					new_cs = to_custom_chord(c)
				else:
					cs = harmony.chordSymbolFromChord(c)
					if "Cannot" in cs.figure:
						pitches = [p for p in c.pitches]
						for idx, p in enumerate(pitches):
							if p.accidental is not None and "#" in p.name:
								pitches[idx] = pitch.Pitch(note_replacements[p.name], octave=p.octave)
						print(pitches)
						new_cs = harmony.chordSymbolFromChord(chord.Chord(pitches))
						measure.append(new_cs.figure)
					else:
						if f"add{cs.figure[0]}#" in cs.figure:
							nc = cs.figure.replace(f"add{cs.figure[0]}#", "b9")
							measure.append(nc)
						else:
							measure.append(cs.figure)
		chord_measures.append(measure)
	return chord_measures

def gen_notes(notes_part):
	note_measures = []
	for item in notes_part:
		measure = []
		for n in item:
			if "note.Note" in str(n) or "note.Rest" in str(n):
				n_name = n.nameWithOctave if "note.Note" in str(n) else "RS"
				n_len = n.quarterLength
				measure.append([n_name, n_len])
		note_measures.append(measure)
	return note_measures

def parse_midi_in_c(file_path):
	song = converter.parse(file_path)
	original_key = song.flatten().getElementsByClass(key.KeySignature)[0]
	if original_key.tonic == "C" and original_key.mode == "major":
		return song
	target_key = key.Key("C") if original_key.mode == "major" else key.Key("A", "minor")
	i = interval.Interval(original_key.tonic, target_key.tonic)

	new_song = song.transpose(i)
	return new_song

Let's see if this worked.

In [34]:
test_song = parse_midi_in_c("data/caravan.midi")
print(test_song.flatten().getElementsByClass(key.KeySignature)[0])

a minor


Looks like it's working as it should. The `test_song` I loaded was `"data/caravan.midi"` and its original key is in F minor/Ab major and you can see from the second line that the key was successfully converted to A minor/C major.

Let's see how it handles our note script.

In [35]:
test_notes = gen_notes(test_song.parts[1])
pprint(test_notes[3])

[['D5', 1.0], ['E5', 1.0], ['G#5', 1.0], ['B4', 1.0]]


I'm going to now import `tqdm` to show our loading process for importing songs.

In [36]:
from tqdm.notebook import tqdm

In [115]:
data_path = Path("data")
files = [f for f in data_path.iterdir() if f.is_file()]

NAME_WIDTH = 30
all_songs = [
  [gen_chords(parse_midi_in_c(fp).parts[0]) for fp in tqdm(files, unit="file")],
  [gen_notes(parse_midi_in_c(fp).parts[1]) for fp in tqdm(files, unit="file")]
]

  0%|          | 0/153 [00:00<?, ?file/s]

[<music21.pitch.Pitch F4>, <music21.pitch.Pitch A-4>, <music21.pitch.Pitch C5>, <music21.pitch.Pitch E-5>]
[<music21.pitch.Pitch F4>, <music21.pitch.Pitch A-4>, <music21.pitch.Pitch C5>, <music21.pitch.Pitch E-5>]
[<music21.pitch.Pitch F4>, <music21.pitch.Pitch A-4>, <music21.pitch.Pitch C5>, <music21.pitch.Pitch E-5>]
[<music21.pitch.Pitch F4>, <music21.pitch.Pitch A-4>, <music21.pitch.Pitch C5>, <music21.pitch.Pitch E-5>]
[<music21.pitch.Pitch F4>, <music21.pitch.Pitch A-4>, <music21.pitch.Pitch C5>, <music21.pitch.Pitch E-5>]
[<music21.pitch.Pitch F4>, <music21.pitch.Pitch A-4>, <music21.pitch.Pitch C5>, <music21.pitch.Pitch E-5>]
[<music21.pitch.Pitch B-4>, <music21.pitch.Pitch D-5>, <music21.pitch.Pitch F5>, <music21.pitch.Pitch A-5>]
[<music21.pitch.Pitch E-4>, <music21.pitch.Pitch G4>, <music21.pitch.Pitch B-4>, <music21.pitch.Pitch D-5>]
[<music21.pitch.Pitch E-4>, <music21.pitch.Pitch G4>, <music21.pitch.Pitch B-4>, <music21.pitch.Pitch D-5>]
[<music21.pitch.Pitch A-4>, <music

AccidentalException: m/caddb- is not a supported accidental type

In [38]:
pprint(all_songs[0][3])

[['E5', 2.0], ['C5', 1.0], ['A4', Fraction(2, 3)], ['F4', Fraction(1, 3)]]


Lovely. Our notes script works exactly as anticipated with lists stacked like so:

- List of `all_songs`
  - List of all measures of a song
    - List of all data in the measure
      - List of note/rest information where `["note-name", float(note_len)]`

Now, let's see if we can repair our `gen_chords` function. Right now, `music21` isn't recognizing `alt7` chords.

### Fixing `gen_chords`

`music21`'s `harmony` has `addNewChordSymbol` functionality, let's see if we can get it to recognize an `alt7` chord. The culprit was a B7alt chord with the notes `["B4", "D5", "F5", "A5", "C6", "E-6", "G6"]`.

In [39]:
# harmony.addNewChordSymbol('alt7', '1,b3,b5,b7,b9,b11,b13', ['alt7', '7alt'])
harmony.addNewChordSymbol('7alt', '1,-3,-5,-7,-9,-11,-13', ['alt7', '7alt'])
custom_cs = harmony.ChordSymbol('Balt7')
custom_cs.pitches

(<music21.pitch.Pitch B2>,
 <music21.pitch.Pitch C3>,
 <music21.pitch.Pitch D3>,
 <music21.pitch.Pitch E-3>,
 <music21.pitch.Pitch F3>,
 <music21.pitch.Pitch G3>,
 <music21.pitch.Pitch A3>)

In [113]:
# Define custom chord symbols
harmony.addNewChordSymbol("7alt", "1,-3,-5,-7,-9,-11,-13", ["alt7", "7alt"])
harmony.addNewChordSymbol("m6", "1,-3,5,6", ["m6", "(min6)"])
harmony.addNewChordSymbol("13", "1,3,5,7,9,13", ["13", "(13)"])
harmony.addNewChordSymbol("69", "1,3,5,6,9", ["69", "6/9"])

test = harmony.ChordSymbol("A-13x")
pprint(test.pitches)

custom_chords = ["7alt", "m6", "13", "69"]

# Input chord
input_notes = [note.Note(n) for n in ["A-4", "C5", "E-5", "G-5", "B-5", "F6"]]
input_chord = chord.Chord(input_notes)

root = input_chord.pitches[0].name
custom_idx = [False] * len(custom_chords)

for idx, c_name in enumerate(custom_chords):
    c_chord = harmony.ChordSymbol(f"{root}{c_name}")
    # Check if all input pitches are in this custom chord
    if all(p in c_chord.pitches for p in input_chord.pitches):
        custom_idx[idx] = True

# Print matched chords
for idx, matched in enumerate(custom_idx):
    if matched:
        new_chord = harmony.ChordSymbol(f"{root}{custom_chords[idx]}")
        print(f"Matched chord: {new_chord.figure}")


(<music21.pitch.Pitch A-2>,
 <music21.pitch.Pitch B-2>,
 <music21.pitch.Pitch C3>,
 <music21.pitch.Pitch E-3>,
 <music21.pitch.Pitch F3>,
 <music21.pitch.Pitch G3>)


In [41]:
note_names = ["B4", "D5", "F5", "A5", "C6", "E-6", "G6"]
note_vals = [note.Note(n, quarterLength=1.0) for n in note_names]
test_chord = chord.Chord(note_vals)

# [p.name for p in test_chord.pitches]

def to_alt7(input_chord):
  input_chord_root = input_chord.pitches[0].name
  return harmony.ChordSymbol(f"{input_chord_root}alt7")

def is_alt7(input_chord):
  input_chord_root = input_chord.pitches[0].name
  alt7_chord = harmony.ChordSymbol(f"{input_chord_root}alt7")
  is_in_alt7 = True
  input_pitches = [p.name for p in input_chord.pitches]
  for p in input_pitches:
    if p not in [a.name for a in alt7_chord.pitches]:
      is_in_alt7 = False
  return is_in_alt7

if is_alt7(test_chord):
  updated_chord = to_alt7(test_chord)

print(updated_chord)

<music21.harmony.ChordSymbol Balt7>
